In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

from activation import *
from loss import *
from linear import *
from utils import * 